## Redes Neuronales con PyTorch:


Las redes neuronales se construyen utilizando el paquete **torch.nn**

Teniendo una idea de autograd, **nn** depende de este paquete para definir los modelos y diferenciarlos. Un **nn.Module** contiene capas y un método **forward(input)** que devuelve la salida (output).

##### Un procedimiento de entrenamiento típico para una red neuronal es el siguiente:

 - Definir la red neuronal que tiene algunos parámetros aprendibles (pesos/weights)
 - Iterar sobre un conjunto de datos de entrada (inputs)
 - Procesar la entrada(input) a través de la red
 - Calcular la pérdida o loss (qué tan lejos estan los outputs de ser correctos)
 - Propagar los gradientes de vuelta a los parámetros de la red
 - Actualizar los pesos (weights) de la red, generalmente usando una reglas de actualizacion: **weigth = weight - learning_rate * gradient**

### 1. Definiendo una Red

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 canal imagen input, 6 canales output, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # una operacion afín: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max agrupacion sobre una ventana (2, 2)
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # Si el tamaño es un cuadrado, solo se puede especificar un número
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # todas las dimensiones excepto la dimension del batch
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


**torch.nn** solo admite mini lotes (batches). Todo el paquete **torch.nn** solo permite entradas que son un mini lote de muestras y NO una sola muestra. Por ejemplo en **nn.Conv2d** se tomará un tensor 4D de ``nSamples x nChannels x Height x Width``. Si tiene una sola muestra, simplemente se usa ``input.unsqueeze(0)`` para agregar una dimensión de lote falsa.

Solo tiene que definir la función **forward**. La función **backward** (donde se calculan los gradientes) se define automáticamente mediante **autograd**. Puede usar cualquiera de las operaciones de **Tensor** en la función **forward**

Los parámetros que se pueden aprender de un modelo son devueltos por **net.parameters()**

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


Ahora probemos con una entrada aleatoria de 32x32. 

A Tener en cuenta: El tamaño de entrada esperado para esta red (LeNet) es 32x32. Por ejemplo para usar esta red en el conjunto de datos MNIST, cambie el tamaño de las imágenes del conjunto de datos a 32x32.

In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0098, -0.0274, -0.0671, -0.0148,  0.0986,  0.0164, -0.1319, -0.1066,
         -0.0745, -0.1136]], grad_fn=<AddmmBackward>)


Ahora **Zero** los **gradient buffers** de todos los parámetros y **backprops** con gradientes aleatorios:

In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

#### Para que todo quede mas claro:

- **torch.Tensor** - una array multidimensional con soporte para operaciones con autograd como lo puede ser **backward()**. También contiene el gradiente w.r.t. del tensor

- **nn.Module** - Módulo como tal de las redes neuronales. Manera conveniente para encapsular parámetros con asistentes para moverlos a GPU, exportar, cargar, etc.

- **nn.Parameter** - un tipo de tensor que se registra automáticamente como parámetro cuando se asigna como atributo a un modulo.

- **autograd.Function** - implementa definiciones hacia adelante(forward) y hacia atrás(backward) de una operación autograd. Cada operación de **Tensor** crea al menos un solo nodo de funcion que se conecta a las funciones que crearon un Tensor y codifica su historial.

## 2. Funcion de pérdida o Loss Function

Una función de pérdida toma el par (output, target) de un input y calcula un valor que estima qué tan lejos está el output del target.

Hay varias funciones de pérdida diferentes en el paquete **nn**. Una pérdida simple es: **nn.MSELoss**, que calcula el error cuadrático medio entre la entrada (input) y el objetivo (target).

In [6]:
output = net(input)
target = torch.randn(10)  # a un target cualquiera(random)
target = target.view(1, -1)  # que tenga la misma forma (shape) del output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7860, grad_fn=<MseLossBackward>)


Ahora si sigue la pérdida en la dirección hacia atrás, utilizando su atributo **.grad_fn** verá un gráfico de cálculos similar a este:

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

Entonces cuando llamamos a **loss.backward()** todo el gráfico diferencia la pérdida w.r.t., y todos los Tensores en el gráfico que tiene **require_grad=True** tendrán su Tensor **.grad** acumulado con el gradiente.

In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

## 3. Backprop (propagación de gradientes)

Para "backpropagar" el error todo lo que tenemos que hacer es **loss.backward().** Sin embargo, se deben borrar los gradientes existentes; de lo contrario, los gradientes se acumularán con los ya existentes.

Ahora llamaremos a **loss.backward()** y echaremos un vistazo a los gradientes **bias** de conv1 antes y después del **backward.**

In [8]:
net.zero_grad()     # zeroes los gradientes buffers de todos los parametros

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0170,  0.0029, -0.0093, -0.0015, -0.0016,  0.0023])


Asi tambien es que se usan las funciones de pérdida.

El paquete de redes neuronales(nn) contiene varios módulos y funciones de pérdida que forman los componentes básicos de las redes neuronales profundas. Una lista completa con documentación está aquí <https://pytorch.org/docs/nn>_.

## 4. Actualización de pesos(weights)

La regla de actualización más simple utilizada en la práctica es el **descenso de gradiente estocástico (SGD):**

``weight = weight - learning_rate * gradient``

Esto se puede implementar con simple codigo Python

Sin embargo, a medida que se implementan mas redes neuronales es mejor usar varias reglas de actualización diferentes como SGD, Nesterov-SGD, Adam, RMSProp, etc. Para habilitar esto, creamos un paquete pequeño: **torch.optim** que implementa todos estos métodos.

In [9]:
import torch.optim as optim

# cree su optimizador
optimizer = optim.SGD(net.parameters(), lr=0.01)

# en su loop de entreno:
optimizer.zero_grad()   # zero los gradientes buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Haga la actualizacion


A partir de estos enunciados que repasamos se pueden crear bases solidas para seguir adentrandose en el mundo de las redes neuronales. Es recomendable ver los distintos tipos de NN y como se implementarian cada uno de los conceptos repasados.